In [ ]:
%run utils/experiment_chooser.ipynb
experiment_chooser.choose()

In [ ]:
import tifffile
import numpy as np
import pathlib

from pystackreg import StackReg
from collections import defaultdict
from multiprocessing import Pool

CHANNEL = "Cy5"

experiment_base = experiment_chooser.fetch_base()
processed_imgs = experiment_base / "processed_imgs"
if not processed_imgs.exists():
    processed_imgs.mkdir()
stacked_output = pathlib.Path("/scratch") / "stacked"
if not stacked_output.exists():
    stacked_output.mkdir()

def key(path):
    _, _, idx = path.name.split(".")[0].split("_")
    return int(idx)
    
def stack(paths):
    img = np.array([tifffile.imread(path) for path in sorted(paths, key=key)])
    sr = StackReg(StackReg.RIGID_BODY)
    registered = sr.register_transform_stack(img, reference="first")
    tifffile.imwrite(stacked_output / path.name, registered)
    
inputs = list((experiment_base / "raw_imgs" / CHANNEL / "T1").glob("*/*.tif"))
groups = defaultdict(list)

for path in inputs:
    vertex = path.name.split("_")[0]
    groups[vertex].append(path)
    
try:
    from notebooks.config.prod import CPUS
except:
    CPUS = None

with Pool(CPUS) as p:
    for idx, _ in enumerate(p.imap(stack, groups.values())):
        print("\r", end="")
        print(f"Stacked {idx + 1} / {len(groups)}", end="")